In [1]:
import networkx as nx
import sys
import numpy as np
from abc import ABC, abstractmethod
from collections import namedtuple
from copy import deepcopy
from functools import reduce
from typing import Callable, Any
from itertools import accumulate

In [2]:
from systemflow.node import *

In [5]:
class PositionSample(Mutate):
    k_position = "position (mm,mm)"
    k_last_position = "last position (mm,mm)"
    k_move_rate = "move rate (mm/s)"
    k_dwell_time = "dwell time (s)"

    def __init__(self, name: str = "PositionSample", relevancy_f: Callable = lambda x: 1.0):    
        #list the message fields and properties required for the transform
        fields = []
        properties = []
        #list the host parameters required for the transform
        parameters = [self.k_position,
                    self.k_move_rate,
                    self.k_dwell_time,]
        super().__init__(name, fields, properties, parameters)
        self.relevancy_f = relevancy_f

    def transform(self, message: Message, component: Component) -> tuple[dict, dict, dict]:
        #predict the relevancy of the collected data based on position
        position = component.parameters[self.k_position]
        relevancy = self.relevancy_f(self.k_position)
        msg_fields = {"relevancy (u)": relevancy,
                    self.k_position: position,}

        #predict the latency
        if self.k_last_position not in component.parameters.keys():
            component.parameters[self.k_last_position] = position

        x_vec = component.parameters[self.k_position][0] - component.parameters[self.k_last_position][0]
        y_vec = component.parameters[self.k_position][1] - component.parameters[self.k_last_position][1]
        distance = np.linalg.norm([x_vec, y_vec])
        movement_time = distance / component.parameters[self.k_move_rate]
        latency = movement_time + component.parameters[self.k_dwell_time]

        component.parameters[self.k_last_position] = position

        host_props = {"latency": latency}
        return msg_fields, {}, host_props


In [6]:
ps = PositionSample()

In [7]:
msg0 = Message({}, {})

In [8]:
sample_stage = Component("Sample Stage",
                    [PositionSample(),],
                    {"position (mm,mm)": [0.0, 0.0],
                     "move rate (mm/s)": 100,
                     "dwell time (s)": 1e-3,},
                     {})

In [9]:
msg1, _ = ps(msg0, sample_stage)

In [10]:
class CollectImage(Mutate):
    def __init__(self, name: str = "CollectImage"):
        fields = []
        properties = []
        parameters = ["resolution x", "resolution y", "bit depth", "sample rate"]
        super().__init__(name, fields, properties, parameters)

    def transform(self, message: Message, component: Component) -> tuple[dict, dict, dict]:
        #access the required fields/properties/parameters
        n_px_x = component.parameters["resolution x"]
        n_px_y = component.parameters["resolution y"]
        resolution = (n_px_x,
                      n_px_y,
                      component.parameters["bit depth"],)
        n_bytes = np.prod(resolution) / 8.0
        sample_rate = component.parameters["sample rate"]
        time = 0.0
        sensor_power = component.parameters["pixel energy"] * n_px_x * n_px_y

        #create the new fields in the message

        msg_fields = {"image data": n_bytes,
                  "time": time}
        msg_props = {"resolution": resolution,
                     "sample rate": sample_rate,}

        #create the new properties in the host
        host_props = {"sensor power": sensor_power,}

        return msg_fields, msg_props, host_props
       

In [11]:
ci = CollectImage()

In [12]:
ci_host = Component("Image sensor",
                    [CollectImage(),],
                    {"resolution x": 6000,
                     "resolution y": 4000,
                     "bit depth": 8,
                     "sample rate": 1000,
                     "pixel energy": 1e-6,},
                     {})

In [13]:
msg2, _ = ci(msg1, ci_host)

In [14]:
msg2

Message(fields={'relevancy (u)': 1.0, 'position (mm,mm)': [0.0, 0.0], 'image data': np.float64(24000000.0), 'time': 0.0}, properties={'resolution': (6000, 4000, 8), 'sample rate': 1000})